In [1]:
#!pip install transformers
#!pip install torchaudio
#!pip install soundfile
#!pip install scikit-learn
#!pip install torch

     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ----------------- -------------------- 20.5/43.6 kB 682.7 kB/s eta 0:00:01
     -------------------------------------- 43.6/43.6 kB 725.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   -------- ------------------------------- 2.0/9.3 MB 42.5 MB/s eta 0:00:01
   ------------------ --------------------- 4.4/9.3 MB 55.9 MB/s eta 0:00:01
   ----------------------------------- ---- 8.2/9.3 MB 58.3 MB/s eta 0:00:01
   ---------------------------------------- 9.3/9.3 MB 49.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/402.6 kB ? eta -:--:--
   --------------------------------------- 402.6/402.6 kB 12.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/138.7 kB ? eta -:--:--
   ---------------------------------------- 138.7/138.7 kB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/289.4 kB ? eta -:--:--
   ---------------


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ----------- ---------------------------- 0.7/2.4 MB 14.4 MB/s eta 0:00:01
   ----------------------------------- ---- 2.1/2.4 MB 22.2 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 21.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/159.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/159.7 MB 6.1 MB/s eta 0:00:27
   ---------------------------------------- 0.7/159.7 MB 7.7 MB/s eta 0:00:21
   ---------------------------------------- 2.0/159.7 MB 13.9 MB/s eta 0:00:12
   - -------------------------------------- 4.4/159.7 MB 23.4 MB/s eta 0:00:07
   - -------------------------------------- 7.3/159.7 MB 31.3 MB/s eta 0:00:05
   -- ------------------------------------- 10.2/159.7 MB 36.4 MB/s eta 0:00:05
   --- ------------------------------------ 13.6/159.7 MB 65.6 MB/s eta 0:00:03
   ---- ----------------------------------- 16.5/159.7 MB 65.6 MB/s eta 0:00


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.0 MB 660.6 kB/s eta 0:00:02
   -------------------------------- ------- 0.8/1.0 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 9.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import soundfile as sf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, Dataset
import numpy as np

c:\Users\Jws\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 사용자 정의 데이터셋 클래스
class DepressionDataset(Dataset):
    def __init__(self, audio_files, labels, processor, max_length=16000):
        self.audio_files = audio_files
        self.labels = labels
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio_file = self.audio_files[idx]
        label = self.labels[idx]

        speech, sample_rate = sf.read(audio_file)
        if len(speech) > self.max_length:
            speech = speech[:self.max_length]

        input_values = self.processor(speech, sampling_rate=sample_rate, return_tensors="pt", padding="max_length", max_length=self.max_length).input_values
        return input_values.squeeze(), torch.tensor(label)

# 음성 파일과 라벨 리스트
audio_files = ["path_to_audio_1.wav", "path_to_audio_2.wav", ...]
labels = [0, 1, ...]  # 0: 비우울증, 1: 우울증

# 데이터셋 분할
train_files, test_files, train_labels, test_labels = train_test_split(audio_files, labels, test_size=0.2, random_state=42)

# 모델과 프로세서 로드
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# 데이터셋 및 데이터로더 준비
train_dataset = DepressionDataset(train_files, train_labels, processor)
test_dataset = DepressionDataset(test_files, test_labels, processor)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)

# 간단한 분류 모델 정의 (예: LSTM)
class DepressionClassifier(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DepressionClassifier, self).__init__()
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # 마지막 LSTM 출력 사용
        x = self.fc(x)
        return x

# 모델 초기화
classifier = DepressionClassifier(input_dim=768, hidden_dim=128, output_dim=2)  # wav2vec2.0의 출력 특징 차원은 768입니다.
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    classifier.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        with torch.no_grad():
            features = model(inputs).logits
        outputs = classifier(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# 평가
classifier.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        features = model(inputs).logits
        outputs = classifier(features)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

# 결과 출력
print(classification_report(all_labels, all_preds, target_names=["Non-Depressed", "Depressed"]))


In [23]:
# 음성 파일 경로
audio_file = "../영어데이터/test.wav"

# 음성 파일 로드
speech, sample_rate = sf.read(audio_file)

# 모델과 프로세서 로드
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# 음성 데이터를 모델 입력 형식으로 변환
input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values

# 모델 추론 수행
with torch.no_grad():
    logits = model(input_values).logits

# 추론 결과를 텍스트로 변환
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

print("Transcription: ", transcription[0])


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Transcription:  IF I WERE RUNNING FOR TO COSS PRESIDENT I WOULD VALUE TE LITTERSHIP UM THAT IS COMING FROM THE OBSERVATION AND ATTEMPDT TO PERSONALITY TO TAKE CARE OF PERSON AROUND YOU IF IT WERE THE CASE OF PRESIDENCE OF THE COMPANY THAT MAY BE THERE MIGHT BE DIFFERENT QUALITIES THE SINCE ITS RUNNING FOR TO CLOSS PRESIDENT AT SCHOOL I WILL USED A STRATEGY TO TELL THE COSSMATES THAT I WILL MAKE SURE EVERY ONE IS DOING O KAY SINCE NOWADAYS UM DOE SERIOUSNESS OF SCHOOL VIOLENCE ISSUE HAS BEEN A RISON AND GROWING MORE AND MORE SO I HOPE THE COSS PRESIDENCE OM SHOULD BE SOME ONE WHO CAN MAKE SURE NO ONE FEELS LEFT OG OR BULLIT AT SCHOOL SO I WILL TRY TO UM PURSUIT CLOSSMATES THAT I AM THE RIHT PERSON FOR DOING THAT ALINE WAS THIS I WILL SAY THAT I WILL TRY TO LISTEN TO WHAT OUR COSSMATES REALELY WANT AND TRY TO REFLECT THAT BY HAVING CONVERSATIONS WITH THE SCHOOL ADMINISTRATION OR TEACHERS WITH MY COMMUNICATION SKILLS FOR EXAMPLE I THINK SUDENTS UM OFTEN CARE A LOT ABOUT SCHOOL FESTIBLES A

In [22]:
# 음성 파일 경로
audio_file = "../영어데이터/test.wav"

# 음성 파일 로드
speech, sample_rate = sf.read(audio_file)

# 모델과 프로세서 로드
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')

# 음성 데이터를 모델 입력 형식으로 변환
input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values

# input_data GPU로 옮김
input_values = input_values.cuda()

# 모델 추론 수행
with torch.no_grad():
    logits = model(input_values).logits

# 추론 결과를 텍스트로 변환
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

print("Transcription: ", transcription[0])


Some weights of the model checkpoint at kresnik/wav2vec2-large-xlsr-korean were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at kresnik/wav2vec2-large-xlsr-korean and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRA

Transcription:  사이볼가닝서리가 스퍼지른 아이 바리 놀러를 시  에스커닝프함비 아스 베이쉬 에마 트 벌신나 리트 딜기어 훨생을 시일스월  이 타세랜 우스 카함 페니 단맥리 틀 마이비를 과리신 에트바닝소를과스 퍼세된제아치고 아이고 니스첫 시발를 기이트차오백 가스메이 트라오 맥 알의 원에 수도 이노게신스나와네이너 시를 스는서치고 마영은 시시 하스패너바이스 행프호인모임모 아이호가스퍼랜된 치함 십 이성원 킹 맥 주열 명원 디어 스라스더와 골고의 아치고 셔아와 이츠 아 벌씨에 가스메이 아이의안도바 이할 수 벌종인어와인 웨스 사고 세이다라고 주차료 센토그라와 과스이 주별 의 이한 좌출할 분하가가하인 카별세 신스부터스코 알이면네주에쉬할 어티  부말 균미이 게쉬스 없서기지상호 아이딩스된 업둔기 마르 바우고 라스스 퍼센트어 투입하와 맥주에 달리 아기학빠 에린 기리 반투발 아 얼 다시 스맥을 이번도 바이스콜 라이마오 개너이 신 스 슐겐 에마맥셔할필요한 화련성 먹대 사티 되리요


In [19]:
import torch
print(torch.cuda.is_available())  # True가 출력되면 CUDA를 사용할 수 있는 것입니다.

False


In [17]:
#!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu125


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu125


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch
print(torch.cuda.is_available())  # True가 출력되면 CUDA를 사용할 수 있는 것입니다.
print(torch.cuda.get_device_name(0))  # 설치된 GPU의 이름을 출력합니다.

True
NVIDIA GeForce MX450


In [2]:
#!pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.3.1
Uninstalling torch-2.3.1:
  Successfully uninstalled torch-2.3.1
Found existing installation: torchvision 0.18.1
Uninstalling torchvision-0.18.1:
  Successfully uninstalled torchvision-0.18.1
Found existing installation: torchaudio 2.3.1
Uninstalling torchaudio-2.3.1:
  Successfully uninstalled torchaudio-2.3.1


In [4]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB 39.7 MB/s eta 0:01:02
     ---------------------------------------- 0.0/2.4 GB 63.8 MB/s eta 0:00:38
     ---------------------------------------- 0.0/2.4 GB 48.0 MB/s eta 0:00:51
     ---------------------------------------- 0.0/2.4 GB 54.7 MB/s eta 0:00:45
     ---------------------------------------- 0.0/2.4 GB 59.5 MB/s eta 0:00:41
     ---------------------------------------- 0.0/2.4 GB 46.7 MB/s eta 0:00:52
     ---------------------------------------- 0.0/2.4 GB 65.2 MB/s eta 0:00:37
     ---------------------------------------- 0.0/2.4 GB 59.5 MB/s eta 0:00:41
     ---------------------------------------- 0.0/2.4 GB 65.6 MB/s eta 0:00:37
     ---------------------------------------- 0.0/2.4 GB 65.6 MB/s eta 0:00:37
     ---------------------------------------- 0.0/2.4 GB 50.4 MB/s eta 0:00:48
 


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Version:", torch.version.cuda)
    print("PyTorch CUDA Version:", torch.backends.cuda.version())
    print("Device Name:", torch.cuda.get_device_name(0))
else:
    print("CUDA not available")